In [1]:
import bs4 as bs
from bs4 import BeautifulSoup
import requests
import csv
import pandas as pd
import time

from selenium import webdriver
from selenium.webdriver.chrome.options import Options

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait

from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager

from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains

In [2]:
#Chrome
chrome_options = Options()

#opens the website
driver = webdriver.Chrome(ChromeDriverManager().install()) #gets the new version of chrome 
driver.maximize_window()

url = "https://forums.edmunds.com/discussion/2444/general/x/what-car-is-right-for-me-help-me-choose"
driver.get(url)


#function to click next 
def click_next():
    try: 
        element = driver.find_element_by_css_selector("[title*='Next Page']")
        element.click()
    except IndexError: 
        print("Next button not found")



====== WebDriver manager ======
Current google-chrome version is 97.0.4692
Get LATEST chromedriver version for 97.0.4692 google-chrome
Driver [C:\Users\matth\.wdm\drivers\chromedriver\win32\97.0.4692.71\chromedriver.exe] found in cache
<ipython-input-2-6d50d586a1df>:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install()) #gets the new version of chrome


In [3]:
messages_list = []
users_list = []
dates_list = []
nbposts_list = []
roles_list = []

num_pages = 84

for page_count in range(1,num_pages):
    time.sleep(1)    
    page = driver.page_source
    soup = BeautifulSoup(page,'lxml')

    messages = soup.find_all('div', class_='Message userContent')
    user_ids = soup.find_all('a', class_='Username js-userCard')
    dates = soup.find_all('span', class_='MItem DateCreated')
    roles = soup.find_all('span', class_='MItem RoleTitle')
    nbposts = soup.find_all('span', class_='MItem PostCount')
    

    #appending messages 
    for i in messages:
            messages_list.append(i.get_text())

    #appending dates
    for i in dates:
            dates_list.append(i.get_text())
            
    #appending users
    for i in user_ids: 
        users_list.append(i.get_text())
        
    #appending user nb of posts
    for i in nbposts: 
        nbposts_list.append(i.get_text())
    
    #appending member role
    for i in roles: 
        roles_list.append(i.get_text())
    

    print("Done scraping", page_count, "of 84 pages.")
    click_next()
    time.sleep(1)
    
df = pd.DataFrame({"Date": dates_list, "User_Id": users_list, "Message": messages_list, "NumberOfPastPosts" : nbposts_list, "Role" : roles_list})
df.to_csv('data.csv')


Done scraping 1 of 84 pages.


<ipython-input-2-6d50d586a1df>:15: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  element = driver.find_element_by_css_selector("[title*='Next Page']")


Done scraping 2 of 84 pages.
Done scraping 3 of 84 pages.
Done scraping 4 of 84 pages.
Done scraping 5 of 84 pages.
Done scraping 6 of 84 pages.
Done scraping 7 of 84 pages.
Done scraping 8 of 84 pages.
Done scraping 9 of 84 pages.
Done scraping 10 of 84 pages.
Done scraping 11 of 84 pages.
Done scraping 12 of 84 pages.
Done scraping 13 of 84 pages.
Done scraping 14 of 84 pages.
Done scraping 15 of 84 pages.
Done scraping 16 of 84 pages.
Done scraping 17 of 84 pages.
Done scraping 18 of 84 pages.
Done scraping 19 of 84 pages.
Done scraping 20 of 84 pages.
Done scraping 21 of 84 pages.
Done scraping 22 of 84 pages.
Done scraping 23 of 84 pages.
Done scraping 24 of 84 pages.
Done scraping 25 of 84 pages.
Done scraping 26 of 84 pages.
Done scraping 27 of 84 pages.
Done scraping 28 of 84 pages.
Done scraping 29 of 84 pages.
Done scraping 30 of 84 pages.
Done scraping 31 of 84 pages.
Done scraping 32 of 84 pages.
Done scraping 33 of 84 pages.
Done scraping 34 of 84 pages.
Done scraping 35 o

In [4]:
import numpy as np
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize

df = pd.read_csv("data.csv")
df = df.drop(['Unnamed: 0'],axis=1)

df

,Date,User_Id,Message,NumberOfPastPosts,Role
0,\nSeptember 2001,pat,\nNeed help choosing your next vehicle? Tell u...,"Posts: 10,421",Member
1,\nSeptember 2001,willow25,"\nI'm buying a ""new"" car by the end of the yea...",Posts: 2,Member
2,\nOctober 2001,dindak,\nBuy a 2001/ 2002 Oldsmobile Intrigue. It's o...,"Posts: 6,632",Member
3,\nOctober 2001,peteri1,\nAm looking at 1 of 4 choices to replace my 9...,Posts: 21,Member
4,\nOctober 2001,mrdetailer,\nGenerally a 3-4 year old car range is a good...,"Posts: 1,118",Member
...,...,...,...,...,...
4146,\nFebruary 2021,kyfdx,\nqbrozen said: show previous quoteskyfdx said...,"Posts: 203,030",Moderator
4147,\nFebruary 2021,backy,\nKamCottage said:Thank you backy. Since I fir...,"Posts: 18,946",Member
4148,\nFebruary 2021,RayeEliza,\nI am doing major research in trying to find ...,Posts: 6,Member
4149,\nFebruary 2021,mlevine,\nI would lean toward subaru. Need to see whic...,Posts: 512,Member


In [5]:
#cleaning
#dates
from datetime import datetime
df['Date'] = df['Date'].map(lambda x: x.lstrip('\n'))
df['Date'] = pd.to_datetime(df['Date'])

#messages
df['Message'] = df['Message'].map(lambda x: x.lstrip('\n'))
#nb posts
df['NumberOfPastPosts'] = df['NumberOfPastPosts'].map(lambda x: x.lstrip('Posts: '))

df

,Date,User_Id,Message,NumberOfPastPosts,Role
0,2001-09-01,pat,Need help choosing your next vehicle? Tell us ...,"10,421",Member
1,2001-09-01,willow25,"I'm buying a ""new"" car by the end of the year ...",2,Member
2,2001-10-01,dindak,Buy a 2001/ 2002 Oldsmobile Intrigue. It's one...,"6,632",Member
3,2001-10-01,peteri1,Am looking at 1 of 4 choices to replace my 99G...,21,Member
4,2001-10-01,mrdetailer,Generally a 3-4 year old car range is a good o...,"1,118",Member
...,...,...,...,...,...
4146,2021-02-01,kyfdx,qbrozen said: show previous quoteskyfdx said:I...,"203,030",Moderator
4147,2021-02-01,backy,KamCottage said:Thank you backy. Since I first...,"18,946",Member
4148,2021-02-01,RayeEliza,I am doing major research in trying to find my...,6,Member
4149,2021-02-01,mlevine,I would lean toward subaru. Need to see which ...,512,Member
